## Data Analysis

This is the first notebook on EDA focusing on data analysis.

We will analyse the dataset to identify:

1. Missing values
2. Numerical variables
3. Distribution of the numerical variables
4. Outliers
5. Categorical variables
6. Cardinality of the categorical variables
7. Potential relationship between the variables and the target

### Code vs Pseudocode

Most of this is actual code.  However, since there is no dataset included, we will use the variable **data** to represent data that has been loaded in.

Other conventions, where needed will use the format **target_col** for the column that we are looking to predict or classify, **num_col** for numerical column, **cat_col** for categorical column, **str_col** for string column, **date_col** for columns containing dates.

In [ ]:
# standard imports

# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt
% matplotlib inline

# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)

In [ ]:
# load dataset
data = pd.read_csv('myFile.csv')

# rows and columns of the data
print(data.shape)

# visualise the dataset
data.head()

In [ ]:
# make a list of the variables that contain missing values
vars_with_na = [var for var in data.columns if data[var].isnull().sum()>1]

# print the variable name and the percentage of missing values
for var in vars_with_na:
    print(var, np.round(data[var].isnull().mean(), 3),  ' % missing values')

### Relationship between values being missing and target_col

Evaluate the **target_col** where the information is missing, for each variable.

In [ ]:
def analyse_na_value(df, var):
    df = df.copy()
    
    # let's make a variable that indicates 1 if the observation was missing or zero otherwise
    df[var] = np.where(df[var].isnull(), 1, 0)
    
    # let's calculate the mean target_col where the information is missing or present
    # for classification, you may want to consider counts of class(es)
    df.groupby(var)['target_col'].median().plot.bar()
    plt.title(var)
    plt.show()
    
    
# execute the plotting function for each variable    
for var in vars_with_na:
    analyse_na_value(data, var)

### Numerical variables

Find out what numerical variables are in the dataset

In [ ]:
# list of numerical variables
num_vars = [var for var in data.columns if data[var].dtypes != 'O']

print('Number of numerical variables: ', len(num_vars))

# visualise the numerical variables
data[num_vars].head()

### Temporal variables

Find out which variables contain dates or years. Typically, we will not use date variables as is, rather we extract information from them. For example, the difference in years between the current time and the event, or the date of another event such as rate change, etc. This can be used in feature engineering.

In [ ]:
# list of variables that contain year information
# you might need to modify the condition below to find YR, YEAR, Yr or Year

year_vars = [var for var in num_vars if 'Yr' in var or 'Year' in var]

year_vars

In [ ]:
# explore the content of these year variables
# look at the spread of values and unique nature of them

for var in year_vars:
    print(var, data[var].unique())
    print()

In [ ]:
# might be able to explore the target_col relationship to years
# this may or may not prove valuable
# if this is classification, perhaps numbers, counts may be helpful

data.groupby('year_col')['target_col'].median().plot()
plt.ylabel('Median target_col)
plt.title('Change in target_col with the years')

In [ ]:
# additional temporal exploration
# this makes a series of scatter plots

# explore the relationship between the year variables and the target_col in a bit of more details
def analyse_year_vars(df, var):
    df = df.copy()
    
    # capture difference between year variable and current year or other date event
    # use whatever 'date_col' is applicable - such as date of loss, etc.
    df[var] = df['date_col'] - df[var]
    
    plt.scatter(df[var], df['target_col'])
    plt.ylabel('target_col')
    plt.xlabel(var)
    plt.show()
    
for var in year_vars:
    if var !='date_col':
        analyse_year_vars(data, var)

### Discrete variables

Find which variables are discrete, i.e., show a finite number of values.  For illustration, the unique count is set to 20, if there are more than 20 values then it's considered continuous.  Set your threshold accordingly.

In [ ]:
#  list of discrete variables
discrete_vars = [var for var in num_vars if len(data[var].unique())<20 and var not in year_vars+['Id']]

print('Number of discrete variables: ', len(discrete_vars))

Often these variables tend to be Qualifications or grading scales, or refer to the number of drivers, or rooms, units, etc. Analyse their contribution to the **target_col**.  For example, do they contribute to the size of a claim?

In [ ]:
# as with others, the metric may differ if you have a classification model

def analyse_discrete(df, var):
    df = df.copy()
    df.groupby(var)['target_col'].median().plot.bar()
    plt.title(var)
    plt.ylabel('target_col')
    plt.show()

    
# this will make a bunch of bar plots
for var in discrete_vars:
    analyse_discrete(data, var)

### Continuous variables

Find the distribution of the continuous variables. Continuous variables are all those that are not temporal or discrete variables in our dataset.

In [ ]:
# list of continuous variables
# Pandas supllis the Id

cont_vars = [var for var in num_vars if var not in discrete_vars+year_vars+['Id']]

print('Number of continuous variables: ', len(cont_vars))

In [ ]:
# analyse the distributions of these variables
# this is designed to get a count of the targets, so the ylabel is modified to reflect that
# this makes histograms, you could try it with box or violin plots

def analyse_continous(df, var):
    df = df.copy()
    df[var].hist(bins=20)
    plt.ylabel('Number of target units')
    plt.xlabel(var)
    plt.title(var)
    plt.show()

# makes a bunch of histograms
for var in cont_vars:
    analyse_continous(data, var)

Depending on your distributions and if there are requirements for Gaussian distributions.  Sometimes, a log transformation, can change/reduce skewness in the distribution.

Let's also evaluate here if a log transformation renders the variables more Gaussian looking

In [ ]:
# analyse the distributions of these transformed variables
def analyse_transformed_continous(df, var):
    df = df.copy()
    
    # log does not take negative values, so be careful and skip those variables
    if 0 in data[var].unique():
        pass
    else:
        # log transform the variable
        df[var] = np.log(df[var])
        df[var].hist(bins=20)
        plt.ylabel('Number of target units')
        plt.xlabel(var)
        plt.title(var)
        plt.show()

# more histogram plots
for var in cont_vars:
    analyse_transformed_continous(data, var)

In [ ]:
# explore the relationship between the target_col and the transformed variables
# with more detail

def transform_analyse_continous(df, var):
    df = df.copy()
    
    # log does not take negative values, be careful and skip those variables
    if 0 in data[var].unique():
        pass
    else:
        # log transform
        df[var] = np.log(df[var])
        df['target_col'] = np.log(df['target_col'])
        plt.scatter(df[var], df['target_col'])
        plt.ylabel('target_col')
        plt.xlabel(var)
        plt.show()
    
# make more scatter plots
for var in cont_vars:
    if var !='target_col':
        transform_analyse_continous(data, var)

### Find the Outliers

In [ ]:
# make boxplots to visualise outliers in the continuous variables
# you could choose violin plots too

def find_outliers(df, var):
    df = df.copy()
    
    # log does not take negative values, so skip those variables
    if 0 in data[var].unique():
        pass
    else:
        df[var] = np.log(df[var])
        df.boxplot(column=var)
        plt.title(var)
        plt.ylabel(var)
        plt.show()
    
for var in cont_vars:
    find_outliers(data, var)

### Categorical Variables

In [ ]:
### Categorical variables

cat_vars = [var for var in data.columns if data[var].dtypes=='O']

print('Number of categorical variables: ', len(cat_vars))

### Number of labels: cardinality

Evaluate how many different categories are present in each of the variables.

In [ ]:
for var in cat_vars:
    print(var, len(data[var].unique()), ' categories')

### Rare labels:

Investigate now if there are labels that are present only in a small number of target

In [ ]:
# find the rare labels

def analyse_rare_labels(df, var, rare_perc):
    df = df.copy()
    tmp = df.groupby(var)['target_col'].count() / len(df)
    return tmp[tmp<rare_perc]

# output the percentage of labels used with the target
for var in cat_vars:
    print(analyse_rare_labels(data, var, 0.01))
    print()

In [ ]:
# plot the relationship between discreet vars and target
# this reuses the analyse_discrete function above to make lots of bar plots

for var in cat_vars:
    analyse_discrete(data, var)